# Učenje RL za uravnoteženje Cartpole

Ta zvezek je del [učnega načrta AI za začetnike](http://aka.ms/ai-beginners). Navdihnjen je bil z [uradnim PyTorch vodičem](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) in [to implementacijo Cartpole v PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

V tem primeru bomo uporabili RL za treniranje modela, ki bo uravnotežil drog na vozičku, ki se lahko premika levo in desno po vodoravni osi. Za simulacijo droga bomo uporabili okolje [OpenAI Gym](https://www.gymlibrary.ml/).

> **Opomba**: Kodo te lekcije lahko zaženete lokalno (npr. iz Visual Studio Code), v tem primeru se bo simulacija odprla v novem oknu. Če kodo izvajate na spletu, boste morda morali narediti nekaj prilagoditev kode, kot je opisano [tukaj](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Začeli bomo s preverjanjem, ali je Gym nameščen:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Zdaj ustvarimo okolje CartPole in si oglejmo, kako delovati v njem. Okolje ima naslednje lastnosti:

* **Prostor akcij** je množica možnih akcij, ki jih lahko izvedemo na vsakem koraku simulacije
* **Prostor opazovanj** je prostor opazovanj, ki jih lahko opravimo


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Poglejmo, kako simulacija deluje. Naslednja zanka izvaja simulacijo, dokler `env.step` ne vrne zaključne zastavice `done`. Naključno bomo izbrali akcije z uporabo `env.action_space.sample()`, kar pomeni, da bo poskus verjetno zelo hitro spodletel (okolje CartPole se zaključi, ko hitrost CartPole, njegova pozicija ali kot presežejo določene meje).

> Simulacija se bo odprla v novem oknu. Kodo lahko zaženete večkrat in opazujete, kako se obnaša.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Opazite lahko, da opazovanja vsebujejo 4 številke. Te so:
- Položaj vozička
- Hitrost vozička
- Kot droga
- Hitrost vrtenja droga

`rew` je nagrada, ki jo prejmemo na vsakem koraku. V okolju CartPole ste nagrajeni z 1 točko za vsak simulacijski korak, cilj pa je maksimizirati skupno nagrado, torej čas, ko lahko CartPole ohranja ravnotežje, ne da bi padel.

Med krepitvenim učenjem je naš cilj trenirati **politiko** $\pi$, ki nam za vsako stanje $s$ pove, katero dejanje $a$ naj izvedemo, torej v bistvu $a = \pi(s)$.

Če želite verjetnostno rešitev, lahko politiko razumete kot vračanje nabora verjetnosti za vsako dejanje, torej $\pi(a|s)$ bi pomenilo verjetnost, da bi morali v stanju $s$ izvesti dejanje $a$.

## Metoda gradienta politike

V najpreprostejšem algoritmu za krepitveno učenje, imenovanem **gradient politike**, bomo trenirali nevronsko mrežo za napovedovanje naslednjega dejanja.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Trenirali bomo mrežo z izvajanjem številnih poskusov in posodabljanjem naše mreže po vsakem poskusu. Določimo funkcijo, ki bo izvedla poskus in vrnila rezultate (tako imenovano **sled**) - vse stanje, akcije (in njihove priporočene verjetnosti) ter nagrade:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Ena epizoda se lahko izvede z neizurjenim omrežjem, pri čemer opazimo, da je skupna nagrada (tj. dolžina epizode) zelo nizka:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Eden iz zahtevnih vidikov algoritma za gradient politike je uporaba **diskontiranih nagrad**. Ideja je, da izračunamo vektor skupnih nagrad na vsakem koraku igre, pri čemer med tem procesom diskontiramo zgodnje nagrade z uporabo nekega koeficienta $gamma$. Prav tako normaliziramo nastali vektor, saj ga bomo uporabili kot utež za vplivanje na naše učenje:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Zdaj pa začnimo z dejanskim treningom! Izvedli bomo 300 epizod, pri vsaki epizodi pa bomo naredili naslednje:

1. Izvedemo eksperiment in zberemo sled.
2. Izračunamo razliko (`gradients`) med izvedenimi akcijami in napovedanimi verjetnostmi. Manjša kot je razlika, bolj smo prepričani, da smo izbrali pravo akcijo.
3. Izračunamo diskontirane nagrade in pomnožimo gradient z diskontiranimi nagradami - to zagotavlja, da koraki z višjimi nagradami bolj vplivajo na končni rezultat kot tisti z nižjimi nagradami.
4. Pričakovane ciljne akcije za naš nevronski model bodo delno izhajale iz napovedanih verjetnosti med izvajanjem in delno iz izračunanih gradientov. Parameter `alpha` bomo uporabili za določitev, v kolikšni meri se gradienti in nagrade upoštevajo - to se imenuje *stopnja učenja* algoritma za okrepitev.
5. Na koncu treniramo naš model na stanjih in pričakovanih akcijah ter ponovimo proces.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Zdaj zaženimo epizodo z upodabljanjem, da vidimo rezultat:


In [ ]:
_ = run_episode(render=True)

Upajmo, da lahko zdaj vidite, da se droga kar dobro uravnoteži!

## Model Actor-Critic

Model Actor-Critic je nadaljnji razvoj politik gradientov, pri katerem zgradimo nevronsko mrežo za učenje tako politike kot ocenjenih nagrad. Mreža bo imela dva izhoda (ali pa si jo lahko predstavljate kot dve ločeni mreži):
* **Actor** bo priporočil dejanje, ki ga je treba izvesti, tako da nam poda porazdelitev verjetnosti stanj, kot v modelu politik gradientov.
* **Critic** bo ocenil, kakšna bi bila nagrada za ta dejanja. Vrne skupne ocenjene nagrade v prihodnosti za dano stanje.

Definirajmo tak model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Morali bi nekoliko spremeniti naši funkciji `discounted_rewards` in `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Zdaj bomo zagnali glavno zanko za učenje. Uporabili bomo ročni postopek učenja mreže z izračunavanjem ustreznih funkcij izgube in posodabljanjem parametrov mreže:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Ključne točke

V tem prikazu smo spoznali dva algoritma za okrepitevno učenje: preprost gradient politike in bolj izpopolnjen igralec-kritik. Vidite lahko, da ti algoritmi delujejo z abstraktnimi pojmi stanja, dejanja in nagrade – zato jih je mogoče uporabiti v zelo različnih okoljih.

Okrepitevno učenje nam omogoča, da se naučimo najboljše strategije za reševanje problema zgolj z opazovanjem končne nagrade. Dejstvo, da ne potrebujemo označenih podatkovnih zbirk, nam omogoča, da simulacije večkrat ponovimo in tako optimiziramo naše modele. Kljub temu pa okrepitevno učenje še vedno prinaša številne izzive, ki jih lahko spoznate, če se odločite poglobiti v to zanimivo področje umetne inteligence.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
